In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from os import listdir
print ('The version of TensorFlow is {}'.format(tf.__version__))

The version of TensorFlow is 1.4.0


In [2]:
root_path = './data'

appended_data = []
for file_name in listdir(root_path):
    file_path = root_path + '/' + file_name.encode().decode('utf-8')
    data_from_one_csv = pd.read_csv(file_path, skiprows=1)
    appended_data.append(data_from_one_csv)
    
data = pd.concat(appended_data, axis=0)
data = data.drop(columns = ['ACN', 'Date', 'Local Time Of Day', 'Ceiling', 'Callback', 'Callback.1', 'Unnamed: 96'])
data = data.rename(index=str, columns={"Flight Phase": "Flight Phase1"})

## drop the rows with empty synopsis description
data = data[pd.notnull(data['Synopsis'])]

X = data.drop(columns = 'Result')
Y_raw = pd.DataFrame(data['Result'])

processed_Y = []
for index, row in Y_raw.iterrows():
    #print (index, row['Result'])
    outcome = row['Result']
    if type(outcome) == np.float:
        res = 'unknown'
        processed_Y.append(res)
    elif ';' in outcome:
        res = str(outcome).split(';')[0]
        processed_Y.append(res)
    else:
        res = outcome
        processed_Y.append(res)

Y = pd.DataFrame(processed_Y, columns = ['Result'])

In [3]:
## compress the number of labels to be predicted --> map result to risk level
rate_nine = ['General Declared Emergency', 'General Physical Injury / Incapacitation', 'Flight Crew Inflight Shutdown', 
             'Air Traffic Control Separated Traffic', 'Aircraft Aircraft Damaged']

rate_seven = ['General Evacuated', 'Flight Crew Landed as Precaution', 'Flight Crew Regained Aircraft Control', 
              'Air Traffic Control Issued Advisory / Alert', 'Flight Crew Landed in Emergency Condition',
              'Flight Crew Landed In Emergency Condition']

rate_five = ['General Work Refused', 'Flight Crew Became Reoriented', 'Flight Crew Diverted', 
             'Flight Crew Executed Go Around / Missed Approach', 
             'Flight Crew Overcame Equipment Problem', 'Flight Crew Rejected Takeoff', 'Flight Crew Took Evasive Action', 
             'Air Traffic Control Issued New Clearance']

rate_three = ['General Maintenance Action', 'General Flight Cancelled / Delayed', 'General Release Refused / Aircraft Not Accepted', 
              'Flight Crew Overrode Automation', 'Flight Crew FLC Overrode Automation',
              'Flight Crew Exited Penetrated Airspace', 
              'Flight Crew Requested ATC Assistance / Clarification', 'Flight Crew Landed As Precaution',
              'Flight Crew Returned To Clearance', 'Flight Crew Returned To Departure Airport',
              'Aircraft Automation Overrode Flight Crew']

rate_one = ['General Police / Security Involved', 'Flight Crew Returned To Gate', 'Aircraft Equipment Problem Dissipated', 
            'unknown', 'Air Traffic Control Provided Assistance',
            'General None Reported / Taken', 'Flight Crew FLC complied w / Automation / Advisory']

Y_ = []
for i in range(Y.shape[0]):
    if Y['Result'][i] in rate_nine:
        Y_.append(5)
    elif Y['Result'][i] in rate_seven:
        Y_.append(4)
    elif Y['Result'][i] in rate_five:
        Y_.append(3)
    elif Y['Result'][i] in rate_three:
        Y_.append(2)
    elif Y['Result'][i] in rate_one:
        Y_.append(1)
    else:
        print (Y['Result'][i])

outcomes = np.asarray(Y_)
Y_true = pd.DataFrame(Y_, index = X.index)
unique, counts = np.unique(outcomes, return_counts=True)

In [4]:
X['Synopsis'].shape[0]

64573

In [5]:
from tensorflow.contrib import learn

max_document_length = max([len(x.split(" ")) for x in X['Synopsis']])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
X_processed = np.array(list(vocab_processor.fit_transform(X['Synopsis'])))
max_document_length

Y_true.replace(to_replace = 1, value = 0, inplace = True)
Y_true.replace(to_replace = 2, value = 0, inplace = True)
Y_true.replace(to_replace = 3, value = 0, inplace = True)
Y_true.replace(to_replace = 5, value = 0, inplace = True)
Y_true.replace(to_replace = 4, value = 1, inplace = True)

n_classes = 2
mat = np.zeros(shape = (Y_true.shape[0], n_classes), dtype = int)

for i in range(Y_true.shape[0]):
    for j in range(n_classes):
        if Y_true[0][i] == j:
            mat[i, j] = 1
            break

In [6]:
Y_true = mat
Y_true

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_processed, Y_true, test_size = 0.2, random_state = 101)

In [8]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 256, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 200, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [9]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()

['-f',
 'C:\\Users\\ZhangXiaoge\\AppData\\Roaming\\jupyter\\runtime\\kernel-74c53d72-956a-49d3-b447-7fcc62a42e82.json']

In [10]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy, batch_predictions = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy, cnn.predictions],
                feed_dict)
            
            #all_predictions = np.concatenate([all_predictions, batch_predictions])
            
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            print(batch_predictions)
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                #dev_step(x_test, y_test, writer=dev_summary_writer)
                print("")
                
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2018-02-19T17:00:04.366441: step 101, loss 0.884799, acc 0.835
2018-02-19T17:00:07.151411: step 102, loss 0.852944, acc 0.85
2018-02-19T17:00:10.183548: step 103, loss 1.06954, acc 0.815
2018-02-19T17:00:13.165651: step 104, loss 1.31154, acc 0.815
2018-02-19T17:00:15.814520: step 105, loss 0.919784, acc 0.84
2018-02-19T17:00:18.607491: step 106, loss 1.05547, acc 0.775
2018-02-19T17:00:21.513542: step 107, loss 0.788099, acc 0.835
2018-02-19T17:00:24.280499: step 108, loss 1.27189, acc 0.76
2018-02-19T17:00:27.125503: step 109, loss 0.762649, acc 0.785
2018-02-19T17:00:29.994527: step 110, loss 0.961137, acc 0.79
2018-02-19T17:00:32.683426: step 111, loss 1.14347, acc 0.78
2018-02-19T17:00:35.504415: step 112, loss 0.907189, acc 0.785
2018-02-19T17:00:38.538557: step 113, loss 0.922004, acc 0.82
2018-02-19T17:00:41.621733: step 114, loss 0.890843, acc 0.81
2018-02-19T17:00:44.635860: step 115, loss 0.951282, acc 0.77
2018-02-19T17:00:47.666998: step 116, loss 1.37611, acc 0.78
2018-02

2018-02-19T17:06:25.529996: step 231, loss 0.808082, acc 0.78
2018-02-19T17:06:28.328970: step 232, loss 0.63819, acc 0.82
2018-02-19T17:06:31.046889: step 233, loss 0.734546, acc 0.775
2018-02-19T17:06:34.078028: step 234, loss 0.534272, acc 0.815
2018-02-19T17:06:36.934044: step 235, loss 0.4807, acc 0.845
2018-02-19T17:06:39.765042: step 236, loss 0.906177, acc 0.81
2018-02-19T17:06:42.722128: step 237, loss 0.740371, acc 0.81
2018-02-19T17:06:45.663204: step 238, loss 0.640824, acc 0.84
2018-02-19T17:06:48.463181: step 239, loss 0.685181, acc 0.775
2018-02-19T17:06:51.518336: step 240, loss 0.534213, acc 0.835
2018-02-19T17:06:54.239256: step 241, loss 0.512461, acc 0.84
2018-02-19T17:06:57.208352: step 242, loss 0.602415, acc 0.85
2018-02-19T17:07:00.189455: step 243, loss 0.492893, acc 0.875
2018-02-19T17:07:02.950403: step 244, loss 0.723521, acc 0.77
2018-02-19T17:07:05.867461: step 245, loss 0.695981, acc 0.84
2018-02-19T17:07:08.851567: step 246, loss 0.816789, acc 0.815
2018

2018-02-19T17:12:25.278378: step 361, loss 0.437508, acc 0.86
2018-02-19T17:12:28.391574: step 362, loss 0.466119, acc 0.855
2018-02-19T17:12:31.296625: step 363, loss 0.412707, acc 0.845
2018-02-19T17:12:34.069581: step 364, loss 0.463221, acc 0.865
2018-02-19T17:12:36.956618: step 365, loss 0.438122, acc 0.845
2018-02-19T17:12:39.885687: step 366, loss 0.479132, acc 0.815
2018-02-19T17:12:42.744705: step 367, loss 0.41415, acc 0.79
2018-02-19T17:12:45.838887: step 368, loss 0.483831, acc 0.84
2018-02-19T17:12:48.923064: step 369, loss 0.337271, acc 0.875
2018-02-19T17:12:51.859135: step 370, loss 0.39788, acc 0.895
2018-02-19T17:12:54.813221: step 371, loss 0.580464, acc 0.805
2018-02-19T17:12:57.832351: step 372, loss 0.343669, acc 0.895
2018-02-19T17:13:00.695370: step 373, loss 0.403594, acc 0.825
2018-02-19T17:13:03.427300: step 374, loss 0.379245, acc 0.865
2018-02-19T17:13:06.262301: step 375, loss 0.479467, acc 0.79
2018-02-19T17:13:09.111309: step 376, loss 0.359274, acc 0.87

2018-02-19T17:18:22.148266: step 491, loss 0.431398, acc 0.87
2018-02-19T17:18:25.078334: step 492, loss 0.418161, acc 0.85
2018-02-19T17:18:27.949360: step 493, loss 0.355333, acc 0.87
2018-02-19T17:18:30.726323: step 494, loss 0.356897, acc 0.875
2018-02-19T17:18:33.465252: step 495, loss 0.367981, acc 0.855
2018-02-19T17:18:36.222198: step 496, loss 0.336292, acc 0.89
2018-02-19T17:18:38.946120: step 497, loss 0.325836, acc 0.895
2018-02-19T17:18:41.708071: step 498, loss 0.343273, acc 0.87
2018-02-19T17:18:44.481027: step 499, loss 0.385357, acc 0.865
2018-02-19T17:18:47.478141: step 500, loss 0.194998, acc 0.96

Evaluation:

Saved model checkpoint to D:\OneDrive - Vanderbilt\Code\NASA_Project\runs\1519080936\checkpoints\model-500

2018-02-19T17:18:54.289950: step 501, loss 0.319237, acc 0.895
2018-02-19T17:18:57.523231: step 502, loss 0.417537, acc 0.875
2018-02-19T17:19:00.518345: step 503, loss 0.391721, acc 0.88
2018-02-19T17:19:03.472428: step 504, loss 0.421491, acc 0.83
2018

...

2018-02-20T08:54:17.899756: step 38233, loss 0.0192029, acc 0.995
2018-02-20T08:54:19.209667: step 38234, loss 0.00370474, acc 1
2018-02-20T08:54:20.526596: step 38235, loss 0.000180129, acc 1
2018-02-20T08:54:21.878550: step 38236, loss 0.000373038, acc 1
2018-02-20T08:54:23.188481: step 38237, loss 0.0182798, acc 0.99
2018-02-20T08:54:24.495956: step 38238, loss 4.52969e-05, acc 1
2018-02-20T08:54:25.802999: step 38239, loss 2.94756e-06, acc 1
2018-02-20T08:54:27.106188: step 38240, loss 3.2454e-05, acc 1
2018-02-20T08:54:28.402104: step 38241, loss 0.000684009, acc 1
2018-02-20T08:54:29.707023: step 38242, loss 0.00079479, acc 1
2018-02-20T08:54:31.027955: step 38243, loss 0.00104544, acc 1
2018-02-20T08:54:32.344886: step 38244, loss 0.000454854, acc 1
2018-02-20T08:54:33.643667: step 38245, loss 0.00249191, acc 1
2018-02-20T08:54:34.944600: step 38246, loss 0.00070796, acc 1
2018-02-20T08:54:36.263520: step 38247, loss 0.000452688, acc 1
2018-02-20T08:54:37.563447: step 38248, los

2018-02-20T08:57:04.528430: step 38359, loss 9.09193e-05, acc 1
2018-02-20T08:57:05.833925: step 38360, loss 0.000179462, acc 1
2018-02-20T08:57:07.132841: step 38361, loss 3.31432e-05, acc 1
2018-02-20T08:57:08.464781: step 38362, loss 0.00012275, acc 1
2018-02-20T08:57:09.796725: step 38363, loss 0.000110196, acc 1
2018-02-20T08:57:11.155680: step 38364, loss 0.0756012, acc 0.985
2018-02-20T08:57:12.496666: step 38365, loss 0.00611973, acc 0.995
2018-02-20T08:57:13.798584: step 38366, loss 0.00278874, acc 1
2018-02-20T08:57:15.092402: step 38367, loss 0.000222169, acc 1
2018-02-20T08:57:16.399325: step 38368, loss 0.000322501, acc 1
2018-02-20T08:57:17.702244: step 38369, loss 0.00127304, acc 1
2018-02-20T08:57:18.995157: step 38370, loss 0.00038753, acc 1
2018-02-20T08:57:20.298505: step 38371, loss 0.00199062, acc 1
2018-02-20T08:57:21.601408: step 38372, loss 0.00261561, acc 1
2018-02-20T08:57:22.923295: step 38373, loss 4.82052e-05, acc 1
2018-02-20T08:57:24.230225: step 38374, l

2018-02-20T08:59:52.171758: step 38485, loss 0.00057914, acc 1
2018-02-20T08:59:53.473678: step 38486, loss 0.0241033, acc 0.995
2018-02-20T08:59:54.781601: step 38487, loss 0.000113442, acc 1
2018-02-20T08:59:56.082519: step 38488, loss 0.0307936, acc 0.995
2018-02-20T08:59:57.382436: step 38489, loss 0.00967627, acc 0.995
2018-02-20T08:59:58.675348: step 38490, loss 0.0549243, acc 0.995
2018-02-20T08:59:59.978041: step 38491, loss 0.000436026, acc 1
2018-02-20T09:00:01.272957: step 38492, loss 0.000211512, acc 1
2018-02-20T09:00:02.582142: step 38493, loss 0.0844759, acc 0.99
2018-02-20T09:00:03.883060: step 38494, loss 0.00261932, acc 1
2018-02-20T09:00:05.184978: step 38495, loss 0.000204542, acc 1
2018-02-20T09:00:06.500907: step 38496, loss 0.00017134, acc 1
2018-02-20T09:00:07.790817: step 38497, loss 0.000996563, acc 1
2018-02-20T09:00:09.094752: step 38498, loss 0.0168428, acc 0.99
2018-02-20T09:00:10.403661: step 38499, loss 0.00479384, acc 0.995
2018-02-20T09:00:11.702578: s

2018-02-20T09:02:41.861987: step 38609, loss 0.000318437, acc 1
2018-02-20T09:02:43.185921: step 38610, loss 0.000244813, acc 1


KeyboardInterrupt: 

In [15]:
batches_test = data_helpers.batch_iter(
            list(zip(x_test, y_test)), FLAGS.batch_size, 1)

In [16]:
for batch in batches_test:
    x_batch, y_batch = zip(*batch)
    dev_step(x_batch, y_batch)
    print (y_batch)

2018-02-20T09:07:31.656361: step 38611, loss 3.44177, acc 0.84
[0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:32.864213: step 38611, loss 3.05915, acc 0.85
[0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:34.025032: step 38611, loss 4.35129, acc 0.84
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
(array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:35.188853: step 38611, loss 3.88001, acc 0.86
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:36.333661: step 38611, loss 3.93473, acc 0.815
[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arr

...

2018-02-20T09:07:50.904944: step 38611, loss 4.55245, acc 0.855
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arr

2018-02-20T09:07:52.031739: step 38611, loss 5.12391, acc 0.83
[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
(array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:53.160535: step 38611, loss 5.62676, acc 0.82
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:54.283328: step 38611, loss 2.77091, acc 0.85
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), arra

2018-02-20T09:07:55.398115: step 38611, loss 5.48665, acc 0.785
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
(array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), arr

In [ ]:
x_test.shape

In [ ]:
12915/64

In [ ]:
Y_true

In [ ]:
index = np.where(Y_true[:,1]==1)

In [ ]:
print('The percentage of category 1 is', len(index[0])/X_processed.shape[0])

In [ ]:
step